# Building a PRS score for each individual in the dataset 
## This PRS score will be based on a previously conducted chronic kidney disease GWAS
### Citation: 
Wuttke, M., Li, Y., Li, M., Sieber, K. B., Feitosa, M. F., Gorski, M., Tin, A., Wang, L., Chu, A. Y., Hoppmann, A., Kirsten, H., Giri, A., Chai, J. F., Sveinbjornsson, G., Tayo, B. O., Nutile, T., Fuchsberger, C., Marten, J., Cocca, M., Ghasemi, S., … Pattaro, C. (2019). A catalog of genetic loci associated with kidney function from analyses of a million individuals. Nature genetics, 51(6), 957–972. https://doi.org/10.1038/s41588-019-0407-x
### Relevant SNPs have already been processed down to 1,443. 

In [ ]:
import os
import pandas as pd
bucket = os.getenv('WORKSPACE_BUCKET')
bucket
import hail as hl
import subprocess
import numpy as np

#### Import the relevant SNPs with their p-value and beta estimate

In [ ]:
snps_file_path = 'gwas_SNPs.csv'

my_bucket = os.getenv('WORKSPACE_BUCKET')

# copy csv file from the bucket to the current working space
os.system(f"gsutil cp '{my_bucket}/data/{snps_file_path}' .")

print(f'[INFO] {snps_file_path} is successfully downloaded into your working space')
# save dataframe in a csv file in the same workspace as the notebook
snp_df = pd.read_csv(snps_file_path)
snp_df.head()


In [ ]:
hl.default_reference(new_default_reference = "GRCh38")

In [ ]:
phenotype_filename = f'{bucket}/data/genomics_ckd_phenotypes.tsv'
phenotype_filename

I am loading the multi-allelic variants split hail matrix table. The split table refers to each allele variant having it's own row. This will allow me to transform the hail matrix table into spark format and then perform a genome-wide association study (GWAS) on my population. After this is complete, I will assemble a polygenic risk score from the results.
"WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH"

In [ ]:
mt_wgs_clinvar_path = os.getenv("WGS_ACAF_THRESHOLD_SPLIT_HAIL_PATH")

In [ ]:
mt = hl.read_matrix_table(mt_wgs_clinvar_path)

In [ ]:
mt.count()

In [ ]:
mt.describe()

In [ ]:
phenotypes_ht = (hl.import_table(phenotype_filename,
                              types={'person_id':hl.tstr},
                              impute=False,
                              key='person_id')
             )

The hail matrix table typically documents the subject id as 's'. Therefore, I will convert my person_id to s. 

In [ ]:
phenotypes_ht = phenotypes_ht.rename({'person_id': 's'})

Now I will join my relevant subject ids with my hail matrix table information.

### Preprocessing the genomic data

Before beginning, we need to anotate the genomic data with the phenotypic data. 

In [ ]:
mt = mt.annotate_cols(pheno = phenotypes_ht[mt.s])

In [ ]:
# Filter individuals to keep only those that are in phenotypes_ht
mt_filtered = mt.filter_cols(hl.is_defined(phenotypes_ht[mt.s]))

In [ ]:
mt_filtered.count()


In [ ]:
mt_filtered.describe()

Converting the SNPS dataframe to a hail matrix table

In [ ]:
gwas_snp_ht = hl.Table.from_pandas(snp_df)

Make sure the chromosome position is in the same format as the patient info hail data

In [ ]:
gwas_snp_ht = gwas_snp_ht.annotate(
    locus=hl.locus('chr' + hl.str(gwas_snp_ht.CHR), gwas_snp_ht.POS)
)

In [ ]:
gwas_snp_ht = gwas_snp_ht.key_by('locus')


In [ ]:
gwas_snp_ht.describe()

In [ ]:
intervals_to_exame = ['chr1', 'chr2', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr15', 'chr16', 'chr17', 'chr18', 'chr21']


In [ ]:
 mt_filtered_2 = hl.filter_intervals(
    mt_filtered,
    [hl.parse_locus_interval(x) for x in intervals_to_exame],
     keep=True)

In [ ]:
mt_filtered_2.show(5)

In [ ]:
mt_filtered_2.describe()

Filtering by interval is more efficient than filtering by row, so making my dataset smaller before filtering by row.

In [ ]:
positions_to_exame = ['chr1:243141745-chr1:243598234', 'chr2:73647508-chr2:217681833', 'chr4:77183300-chr4:81202048', 'chr5:39355591-chr5:176824137', 'chr6:21188017-chr6:160668389', 'chr7:1270699-chr7:156252939', 'chr8:23735559-chr8:23786784', 'chr9:71369556-chr9:71513268', 'chr10:863482-chr10:1171823', 'chr11:30749090-chr11:65555458', 'chr15:39213851-chr15:76151081', 'chr16:20338450-chr16:20430769', 'chr17:19308768-chr17:34959861', 'chr18:24386535-chr18:77160235', 'chr21:16576783-chr21:16578800']


In [ ]:
 mt_filtered_3 = hl.filter_intervals(
    mt_filtered_2,
    [hl.parse_locus_interval(x) for x in positions_to_exame],
     keep=True)

In [ ]:
mt_filtered_3.show(4)

Filtering by the exact SNPs now

In [ ]:
interval_ht = gwas_snp_ht.annotate(
    interval=hl.parse_locus_interval(
        "chr" + hl.str(gwas_snp_ht.CHR) + ":" + 
        hl.str(hl.max(1, gwas_snp_ht.POS - 5)) + "-" + 
        hl.str(gwas_snp_ht.POS + 5),
        reference_genome="GRCh38"
    )
).select("interval")

In [ ]:
mt_filtered_4 = hl.filter_intervals(mt_filtered_3, interval_ht.interval.collect(), keep=True)


In [ ]:
mt_filtered_4.show(4)

In [ ]:
final_snps = mt_filtered_4.filter_rows(hl.is_defined(gwas_snp_ht[mt_filtered_4.locus]))


In [ ]:
final_snps.show(5)

In [ ]:
final_snps.count()

Annotate the filtered data frame with the effect sizes (beta) on CKD for each SNP

In [ ]:
final_snps = final_snps.annotate_rows(beta=gwas_snp_ht[final_snps.locus].BETA)



Calculate the PRS score by multiplying the # of alt alles the participant has by it's effect size on CKD

In [ ]:
final_snps = final_snps.annotate_entries(dosage=final_snps.GT.n_alt_alleles())


In [ ]:
final_snps.describe()

In [ ]:
final_PRS = final_snps.annotate_cols(
    PRS=hl.agg.sum(final_snps.dosage * final_snps.beta)
)

In [ ]:
prs_table = final_PRS.cols().select('PRS')
prs_df = prs_table.to_pandas()


In [ ]:
prs_df.head(10)